In [1]:
# Standard library imports
import json
import random
import sys
import uuid
from datetime import date, datetime, time, timedelta
from pathlib import Path

# Add src to path for imports
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Third-party imports
import folium
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Local imports
from src.database import (
    ClientModel,
    DatabaseManager,
    LocalityModel,
    OrderItemModel,
    OrderModel,
    ProductModel,
    ZoneModel,
)

print("Imports successful!")

Imports successful!


## 1. Database Initialization

Create the SQLite database and all required tables.

In [2]:
# Define paths
DATA_DIR = project_root / "data"
GEO_DIR = DATA_DIR / "geo"
DB_PATH = DATA_DIR / "processed" / "delivery.db"

# Initialize database manager
db = DatabaseManager(DB_PATH)

# Drop existing tables and recreate (fresh start for demo)
db.drop_tables()
db.create_tables()

print(f"Database created at: {DB_PATH}")
print("All tables created successfully!")

Database created at: c:\Users\Santi\Desktop\CV\portafolio\Eco-Bags-Delivery-Optimizer\data\processed\delivery.db
All tables created successfully!


## 2. Load Geographic Data

Load zones and localities from JSON files into the database.

In [3]:
# Load zones
zones_path = GEO_DIR / "zones.json"
zones = db.load_zones_from_json(zones_path)
print(f"Loaded {len(zones)} zones")

# Load localities
localities_path = GEO_DIR / "localities.json"
localities = db.load_localities_from_json(localities_path)
print(f"Loaded {len(localities)} localities")

Loaded 4 zones
Loaded 79 localities


In [4]:
# Verify loaded data
with db.get_session() as session:
    zones_df = pd.read_sql("SELECT * FROM zones", session.bind)
    localities_df = pd.read_sql("SELECT * FROM localities", session.bind)

print("\n=== Zones ===")
display(zones_df)

print(f"\n=== Localities ({len(localities_df)} total) ===")
print(localities_df.groupby("zone_id").size().to_frame("count"))


=== Zones ===


,zone_id,name,color
0,CABA,Buenos Aires City,#FF6B6B
1,NORTH_ZONE,North Zone,#4ECDC4
2,SOUTH_ZONE,South Zone,#45B7D1
3,WEST_ZONE,West Zone,#96CEB4



=== Localities (79 total) ===
            count
zone_id          
CABA           25
NORTH_ZONE     18
SOUTH_ZONE     18
WEST_ZONE      18


## 3. Create Products

Define the 4 product types: 3 main bag types (small, medium, large) and 1 special type.

In [5]:
# Define products
products_data = [
    {
        "product_id": "BAG-SM-250",
        "name": "Eco Bag Small - Pack 250",
        "bag_type": "small",
        "packs_per_pallet": 48,
        "description": "Compact eco-friendly bag, ideal for small purchases",
    },
    {
        "product_id": "BAG-MD-250",
        "name": "Eco Bag Medium - Pack 250",
        "bag_type": "medium",
        "packs_per_pallet": 40,
        "description": "Standard eco-friendly shopping bag, versatile size",
    },
    {
        "product_id": "BAG-LG-250",
        "name": "Eco Bag Large - Pack 250",
        "bag_type": "large",
        "packs_per_pallet": 36,
        "description": "Reinforced eco-friendly bag, for bulk purchases",
    },
    {
        "product_id": "BAG-SP-250",
        "name": "Eco Bag Special Edition - Pack 250",
        "bag_type": "special",
        "packs_per_pallet": 32,
        "description": "Premium custom-printed eco-friendly bag",
    },
]

# Insert products into database
with db.get_session() as session:
    for product_data in products_data:
        product = ProductModel(**product_data)
        session.merge(product)
    session.commit()

# Verify products
with db.get_session() as session:
    products_df = pd.read_sql("SELECT * FROM products", session.bind)

print("=== Products ===")
display(products_df)

=== Products ===


,product_id,name,bag_type,packs_per_pallet,description
0,BAG-SM-250,Eco Bag Small - Pack 250,small,48,"Compact eco-friendly bag, ideal for small purc..."
1,BAG-MD-250,Eco Bag Medium - Pack 250,medium,40,"Standard eco-friendly shopping bag, versatile ..."
2,BAG-LG-250,Eco Bag Large - Pack 250,large,36,"Reinforced eco-friendly bag, for bulk purchases"
3,BAG-SP-250,Eco Bag Special Edition - Pack 250,special,32,Premium custom-printed eco-friendly bag


## 4. Generate Synthetic Clients

Create 30 synthetic clients distributed across all zones with varied attributes.

In [6]:
# Set random seed for reproducibility
random.seed(9)

# Load localities for reference
with open(localities_path, "r", encoding="utf-8") as f:
    localities_dict = json.load(f)

# Argentine business names (realistic Buenos Aires style)
argentine_business_names = [
    "Supermercado Don Pedro",
    "Almacen El Progreso",
    "Distribuidora Los Andes",
    "Comercial San Martin",
    "Mayorista La Union",
    "Autoservicio El Sol",
    "Mercado Central Norte",
    "Fiambreria La Esquina",
    "Supermercado Familiar",
    "Distribuidora del Sur",
    "Almacen y Dietética Vita",
    "Comercio La Estrella",
    "Mayorista Don Juan",
    "Autoservicio Mi Barrio",
    "Mercadito El Vecino",
    "Supermercado Los Amigos",
    "Distribuidora Pampa",
    "Almacen El Rincón",
    "Comercial Rivadavia",
    "Mayorista El Gaucho",
    "Autoservicio La Plaza",
    "Mercado Buenos Aires",
    "Fiambreria Del Centro",
    "Supermercado La Familia",
    "Distribuidora San Jorge",
    "Almacen El Buen Precio",
    "Comercio Don Carlos",
    "Mayorista Del Oeste",
    "Autoservicio El Trebol",
    "Mercadito La Esperanza",
    "Supermercado Norte",
    "Distribuidora La Victoria",
    "Almacen Mi Tierra",
    "Comercial El Puente",
    "Mayorista Santa Fe"
]

# Street names for addresses
street_names = [
    "Av. Corrientes", "Av. Santa Fe", "Av. Rivadavia", "Av. Belgrano",
    "Av. Libertador", "Av. Cabildo", "Av. San Martin", "Av. Juan B. Justo",
    "Av. Callao", "Av. 9 de Julio", "Av. Pueyrredon", "Av. Independencia",
    "Av. Cordoba", "Calle Florida", "Calle Lavalle", "Av. Mitre",
    "Av. Maipu", "Av. Sarmiento", "Av. Alem", "Av. Paseo Colon"
]

# Time windows for deliveries
time_windows = [
    (None, None),  # No restriction
    (time(8, 0), time(12, 0)),  # Morning
    (time(10, 0), time(14, 0)),  # Late morning
    (time(14, 0), time(18, 0)),  # Afternoon
    (time(9, 0), time(17, 0)),  # Business hours
]


def generate_tax_id() -> str:
    """Generate a realistic Argentine CUIT number."""
    prefix = random.choice(["20", "23", "27", "30", "33"])
    middle = str(random.randint(10000000, 99999999))
    suffix = str(random.randint(0, 9))
    return f"{prefix}-{middle}-{suffix}"


def generate_client_id() -> str:
    """Generate a unique client ID."""
    return f"CLI-{uuid.uuid4().hex[:8].upper()}"


# Generate 30 clients
clients_data = []
locality_keys = list(localities_dict.keys())

# Shuffle business names to randomize assignment
shuffled_names = argentine_business_names.copy()
random.shuffle(shuffled_names)

for i in range(30):
    # Select random locality
    locality_key = random.choice(locality_keys)
    locality = localities_dict[locality_key]
    
    # Add small random offset to coordinates (within ~500m)
    lat_offset = random.uniform(-0.005, 0.005)
    lon_offset = random.uniform(-0.005, 0.005)
    
    # Use Argentine business name
    business_name = shuffled_names[i % len(shuffled_names)]
    
    # Generate address
    street = random.choice(street_names)
    number = random.randint(100, 5000)
    address = f"{street} {number}, {locality['name']}, Buenos Aires"
    
    # Select time window
    tw_start, tw_end = random.choice(time_windows)
    
    # Determine client attributes
    is_star = random.random() < 0.2  # 20% star clients
    is_new = random.random() < 0.15  # 15% new clients
    
    # First order date (for non-new clients)
    if is_new:
        first_order = date.today() - timedelta(days=random.randint(1, 30))
    else:
        first_order = date.today() - timedelta(days=random.randint(60, 730))
    
    client = {
        "client_id": generate_client_id(),
        "business_name": business_name,
        "tax_id": generate_tax_id(),
        "billing_address": address,
        "zone_id": locality["zone_id"],
        "latitude": locality["latitude"] + lat_offset,
        "longitude": locality["longitude"] + lon_offset,
        "time_window_start": tw_start,
        "time_window_end": tw_end,
        "is_star_client": is_star,
        "is_new_client": is_new,
        "first_order_date": first_order,
    }
    clients_data.append(client)

# Insert clients into database
with db.get_session() as session:
    for client_data in clients_data:
        client = ClientModel(**client_data)
        session.merge(client)
    session.commit()

print(f"Generated {len(clients_data)} clients")

Generated 30 clients


In [7]:
# Verify clients
with db.get_session() as session:
    clients_df = pd.read_sql("SELECT * FROM clients", session.bind)

print("=== Clients Summary ===")
print(f"Total clients: {len(clients_df)}")
print(f"Star clients: {clients_df['is_star_client'].sum()}")
print(f"New clients: {clients_df['is_new_client'].sum()}")
print(f"\nClients by zone:")
print(clients_df.groupby("zone_id").size().to_frame("count"))

print("\n=== Sample Clients ===")
display(clients_df[["client_id", "business_name", "zone_id", "is_star_client", "is_new_client"]].head(10))

=== Clients Summary ===
Total clients: 30
Star clients: 5
New clients: 8

Clients by zone:
            count
zone_id          
CABA            8
NORTH_ZONE      6
SOUTH_ZONE      6
WEST_ZONE      10

=== Sample Clients ===


,client_id,business_name,zone_id,is_star_client,is_new_client
0,CLI-AE8737DF,Distribuidora La Victoria,NORTH_ZONE,0,0
1,CLI-53DCB6AB,Almacen El Buen Precio,SOUTH_ZONE,0,0
2,CLI-64F893E0,Fiambreria La Esquina,WEST_ZONE,1,0
3,CLI-9E913EF3,Autoservicio El Trebol,WEST_ZONE,0,1
4,CLI-291B2D48,Distribuidora del Sur,WEST_ZONE,1,0
5,CLI-02D2CF44,Autoservicio Mi Barrio,CABA,0,1
6,CLI-3AE16F03,Mayorista La Union,SOUTH_ZONE,0,0
7,CLI-D8C128A2,Supermercado Los Amigos,SOUTH_ZONE,0,1
8,CLI-F9CA9789,Comercial Rivadavia,SOUTH_ZONE,1,0
9,CLI-87A8258E,Mayorista Santa Fe,CABA,0,0


## 5. Generate Synthetic Orders

Create 50 orders with varied attributes for demonstration purposes.

In [8]:
def generate_order_id() -> str:
    """Generate a unique order ID."""
    return f"ORD-{uuid.uuid4().hex[:8].upper()}"


def calculate_pallets(quantity_packs: int, packs_per_pallet: int) -> float:
    """Calculate the number of pallets for a given quantity."""
    return round(quantity_packs / packs_per_pallet, 2)


# Payment status distribution
payment_weights = {
    "paid": 0.35,
    "partial": 0.25,
    "pending": 0.40,
}

# Order status distribution
status_weights = {
    "pending": 0.60,
    "assigned": 0.25,
    "delivered": 0.15,
}

# Get products for reference
with db.get_session() as session:
    products = session.query(ProductModel).all()
    products_info = {p.product_id: p.packs_per_pallet for p in products}

# Generate 50 orders
orders_data = []
order_items_data = []

for i in range(50):
    # Select random client
    client = random.choice(clients_data)
    
    # Generate dates
    issue_date = date.today() - timedelta(days=random.randint(0, 10))
    
    # Delivery deadline based on payment status (will be set after)
    payment_status = random.choices(
        list(payment_weights.keys()),
        weights=list(payment_weights.values())
    )[0]
    
    # Paid orders have shorter deadline (3 days), others 7 days
    if payment_status == "paid":
        deadline_days = random.randint(1, 3)
    else:
        deadline_days = random.randint(3, 7)
    
    delivery_deadline = issue_date + timedelta(days=deadline_days)
    
    # Order status
    order_status = random.choices(
        list(status_weights.keys()),
        weights=list(status_weights.values())
    )[0]
    
    # Is mandatory (10% chance for pending orders)
    is_mandatory = order_status == "pending" and random.random() < 0.1
    
    # Generate order items (1-3 products per order)
    num_items = random.randint(1, 3)
    selected_products = random.sample(list(products_info.keys()), num_items)
    
    order_id = generate_order_id()
    total_pallets = 0
    total_amount = 0
    
    items = []
    for product_id in selected_products:
        packs_per_pallet = products_info[product_id]
        # Order between 0.5 and 3 pallets worth of product
        target_pallets = random.uniform(0.5, 3.0)
        quantity_packs = int(target_pallets * packs_per_pallet)
        quantity_packs = max(10, quantity_packs)  # Minimum 10 packs
        
        pallets = calculate_pallets(quantity_packs, packs_per_pallet)
        total_pallets += pallets
        
        # Price per pack varies by product
        price_per_pack = random.uniform(15, 35)
        total_amount += quantity_packs * price_per_pack
        
        items.append({
            "order_id": order_id,
            "product_id": product_id,
            "quantity_packs": quantity_packs,
            "pallets": pallets,
        })
    
    # Cap total pallets at 8 (truck capacity)
    if total_pallets > 8:
        scale = 7.5 / total_pallets
        total_pallets = 0
        for item in items:
            item["pallets"] = round(item["pallets"] * scale, 2)
            item["quantity_packs"] = int(item["quantity_packs"] * scale)
            total_pallets += item["pallets"]
        total_amount *= scale
    
    order = {
        "order_id": order_id,
        "client_id": client["client_id"],
        "issue_date": issue_date,
        "delivery_deadline": delivery_deadline,
        "delivery_address": client["billing_address"],
        "delivery_latitude": client["latitude"],
        "delivery_longitude": client["longitude"],
        "delivery_zone_id": client["zone_id"],
        "total_amount": round(total_amount, 2),
        "payment_status": payment_status,
        "is_mandatory": is_mandatory,
        "quantity_packs": quantity_packs,
        "total_pallets": round(total_pallets, 2),
        "priority_score": None,  # Will be calculated later
        "status": order_status,
        "created_at": datetime.now() - timedelta(days=random.randint(0, 10)),
    }
    
    orders_data.append(order)
    order_items_data.extend(items)

# Insert orders and items into database
with db.get_session() as session:
    for order_data in orders_data:
        order = OrderModel(**order_data)
        session.merge(order)
    session.commit()
    
    for item_data in order_items_data:
        item = OrderItemModel(**item_data)
        session.add(item)
    session.commit()

print(f"Generated {len(orders_data)} orders with {len(order_items_data)} line items")

Generated 50 orders with 112 line items


In [9]:
# Verify orders
with db.get_session() as session:
    orders_df = pd.read_sql("SELECT * FROM orders", session.bind)
    items_df = pd.read_sql("SELECT * FROM order_items", session.bind)

print("=== Orders Summary ===")
print(f"Total orders: {len(orders_df)}")
print(f"Pending orders: {(orders_df['status'] == 'pending').sum()}")
print(f"Mandatory orders: {orders_df['is_mandatory'].sum()}")
print(f"\nOrders by status:")
print(orders_df.groupby("status").size().to_frame("count"))
print(f"\nOrders by payment status:")
print(orders_df.groupby("payment_status").size().to_frame("count"))
print(f"\nOrders by zone:")
print(orders_df.groupby("delivery_zone_id").size().to_frame("count"))

print("\n=== Sample Orders ===")
display(orders_df[["order_id", "client_id", "delivery_zone_id", "quantity_packs", "total_pallets", "total_amount", "payment_status", "status", "is_mandatory"]].head(10))

=== Orders Summary ===
Total orders: 50
Pending orders: 37
Mandatory orders: 5

Orders by status:
           count
status          
assigned       9
delivered      4
pending       37

Orders by payment status:
                count
payment_status       
paid               18
partial            13
pending            19

Orders by zone:
                  count
delivery_zone_id       
CABA                 13
NORTH_ZONE            3
SOUTH_ZONE           11
WEST_ZONE            23

=== Sample Orders ===


,order_id,client_id,delivery_zone_id,quantity_packs,total_pallets,total_amount,payment_status,status,is_mandatory
0,ORD-506300EE,CLI-FD9DDE4A,WEST_ZONE,26.0,4.06,5268.68,pending,pending,1
1,ORD-CEE59C72,CLI-CA6A13D3,WEST_ZONE,117.0,4.86,5907.45,pending,pending,0
2,ORD-4F690B02,CLI-CA6A13D3,WEST_ZONE,103.0,2.58,2716.33,paid,pending,0
3,ORD-B0365BB7,CLI-13E6F9DC,CABA,75.0,3.94,4764.39,pending,pending,1
4,ORD-53FCC326,CLI-64F893E0,WEST_ZONE,44.0,5.92,6315.58,paid,assigned,0
5,ORD-6B03717F,CLI-E148C0BA,CABA,53.0,1.79,1978.81,pending,pending,0
6,ORD-1F1EDD17,CLI-182B5309,CABA,73.0,7.05,5376.67,pending,assigned,0
7,ORD-01934CFB,CLI-B9701E68,CABA,135.0,7.87,7114.87,paid,pending,0
8,ORD-8E1A15C6,CLI-3AE16F03,SOUTH_ZONE,90.0,2.81,2016.20,pending,pending,0
9,ORD-025343DA,CLI-D81BE990,WEST_ZONE,49.0,5.07,5385.69,partial,pending,1


## 6. Exploratory Queries

Basic SQL queries to explore the data.

In [10]:
# Query: Pending orders with total pallets by zone
query = """
SELECT 
    o.delivery_zone_id as zone,
    COUNT(*) as order_count,
    ROUND(SUM(o.total_pallets), 2) as total_pallets,
    ROUND(AVG(o.total_pallets), 2) as avg_pallets_per_order,
    SUM(CASE WHEN o.is_mandatory = 1 THEN 1 ELSE 0 END) as mandatory_count
FROM orders o
WHERE o.status = 'pending'
GROUP BY o.delivery_zone_id
ORDER BY total_pallets DESC
"""

with db.get_session() as session:
    pending_by_zone = pd.read_sql(query, session.bind)

print("=== Pending Orders by Zone ===")
display(pending_by_zone)

=== Pending Orders by Zone ===


,zone,order_count,total_pallets,avg_pallets_per_order,mandatory_count
0,WEST_ZONE,18,67.70,3.76,2
1,CABA,10,37.75,3.77,2
2,SOUTH_ZONE,6,21.56,3.59,1
3,NORTH_ZONE,3,10.26,3.42,0


In [11]:
# Query: Orders with client details
query = """
SELECT 
    o.order_id,
    c.business_name,
    o.delivery_zone_id as zone,
    o.quantity_packs,
    o.total_pallets,
    o.payment_status,
    o.delivery_deadline,
    CASE WHEN c.is_star_client = 1 THEN 'Yes' ELSE 'No' END as star_client,
    CASE WHEN o.is_mandatory = 1 THEN 'Yes' ELSE 'No' END as mandatory
FROM orders o
JOIN clients c ON o.client_id = c.client_id
WHERE o.status = 'pending'
ORDER BY o.delivery_deadline
LIMIT 15
"""

with db.get_session() as session:
    orders_with_clients = pd.read_sql(query, session.bind)

print("=== Pending Orders with Client Details ===")
display(orders_with_clients)

=== Pending Orders with Client Details ===


,order_id,business_name,zone,quantity_packs,total_pallets,payment_status,delivery_deadline,star_client,mandatory
0,ORD-9D484C47,Autoservicio La Plaza,WEST_ZONE,36.0,1.12,paid,2026-01-07,Yes,No
1,ORD-01934CFB,Almacen Mi Tierra,CABA,135.0,7.87,paid,2026-01-08,No,No
2,ORD-1129F35C,Comercial Rivadavia,SOUTH_ZONE,107.0,4.89,partial,2026-01-09,Yes,No
3,ORD-B0665DC2,Autoservicio El Trebol,WEST_ZONE,57.0,3.88,paid,2026-01-09,No,No
4,ORD-506300EE,Distribuidora Pampa,WEST_ZONE,26.0,4.06,pending,2026-01-10,No,Yes
5,ORD-5275935E,Supermercado Norte,WEST_ZONE,74.0,7.30,pending,2026-01-10,No,No
6,ORD-0BE1B38B,Supermercado Don Pedro,CABA,88.0,2.44,paid,2026-01-10,No,No
7,ORD-770C33DE,Distribuidora La Victoria,NORTH_ZONE,99.0,2.75,pending,2026-01-11,No,No
8,ORD-32F698A9,Distribuidora Los Andes,WEST_ZONE,78.0,4.51,pending,2026-01-11,No,No
9,ORD-CEE59C72,Comercio La Estrella,WEST_ZONE,117.0,4.86,pending,2026-01-12,No,No


In [12]:
# Query: Product popularity
query = """
SELECT 
    p.product_id,
    p.name as product_name,
    p.bag_type,
    COUNT(DISTINCT oi.order_id) as order_count,
    SUM(oi.quantity_packs) as total_packs,
    ROUND(SUM(oi.pallets), 2) as total_pallets
FROM order_items oi
JOIN products p ON oi.product_id = p.product_id
GROUP BY p.product_id
ORDER BY total_pallets DESC
"""

with db.get_session() as session:
    product_popularity = pd.read_sql(query, session.bind)

print("=== Product Popularity ===")
display(product_popularity)

=== Product Popularity ===


,product_id,product_name,bag_type,order_count,total_packs,total_pallets
0,BAG-LG-250,Eco Bag Large - Pack 250,large,31,2029,56.37
1,BAG-SP-250,Eco Bag Special Edition - Pack 250,special,28,1653,51.65
2,BAG-SM-250,Eco Bag Small - Pack 250,small,31,2466,51.34
3,BAG-MD-250,Eco Bag Medium - Pack 250,medium,22,1515,37.89


## 7. Plotly Visualizations

Interactive charts using Plotly.

In [13]:
# Load zone colors for consistent visualization
with open(zones_path, "r", encoding="utf-8") as f:
    zones_colors = json.load(f)

zone_color_map = {zone_id: info["color"] for zone_id, info in zones_colors.items()}

# Get all data for visualization
with db.get_session() as session:
    all_orders_df = pd.read_sql("SELECT * FROM orders", session.bind)
    all_clients_df = pd.read_sql("SELECT * FROM clients", session.bind)

In [14]:
# Chart 1: Orders by Zone (Pending only)
pending_orders = all_orders_df[all_orders_df["status"] == "pending"]
orders_by_zone = pending_orders.groupby("delivery_zone_id").agg({
    "order_id": "count",
    "total_pallets": "sum"
}).reset_index()
orders_by_zone.columns = ["zone", "order_count", "total_pallets"]

fig = px.bar(
    orders_by_zone,
    x="zone",
    y="order_count",
    color="zone",
    color_discrete_map=zone_color_map,
    title="Pending Orders by Zone",
    labels={"order_count": "Number of Orders", "zone": "Zone"},
    text="order_count"
)
fig.update_traces(textposition="outside")
fig.update_layout(
    showlegend=False,
    xaxis_title="Zone",
    yaxis_title="Number of Orders"
)
fig.show()

In [15]:
# Chart 2: Pallets Distribution by Zone
fig = px.pie(
    orders_by_zone,
    values="total_pallets",
    names="zone",
    color="zone",
    color_discrete_map=zone_color_map,
    title="Total Pallets Distribution by Zone (Pending Orders)",
    hole=0.4
)
fig.update_traces(textposition="inside", textinfo="percent+label")
fig.show()

In [16]:
# Chart 3: Orders by Payment Status and Zone
payment_zone = pending_orders.groupby(["delivery_zone_id", "payment_status"]).size().reset_index(name="count")

fig = px.bar(
    payment_zone,
    x="delivery_zone_id",
    y="count",
    color="payment_status",
    title="Pending Orders by Zone and Payment Status",
    labels={"delivery_zone_id": "Zone", "count": "Number of Orders", "payment_status": "Payment Status"},
    barmode="group",
    color_discrete_map={"paid": "#2ecc71", "partial": "#f1c40f", "pending": "#e74c3c"}
)
fig.show()

In [17]:
# Chart 4: Pallets per Order Distribution
fig = px.histogram(
    pending_orders,
    x="total_pallets",
    nbins=20,
    title="Distribution of Pallets per Order (Pending Orders)",
    labels={"total_pallets": "Pallets per Order", "count": "Frequency"},
    color_discrete_sequence=["#3498db"]
)
fig.update_traces(
    marker_line_color="white",
    marker_line_width=1
)
fig.add_vline(x=8, line_dash="dash", line_color="red", annotation_text="Truck Capacity (8 pallets)")
fig.update_layout(xaxis_title="Pallets per Order", yaxis_title="Frequency")
fig.show()

In [18]:
# Chart 5: Client Distribution by Zone with Star/New indicators
client_summary = all_clients_df.groupby("zone_id").agg({
    "client_id": "count",
    "is_star_client": "sum",
    "is_new_client": "sum"
}).reset_index()
client_summary.columns = ["zone", "total_clients", "star_clients", "new_clients"]

fig = go.Figure()

fig.add_trace(go.Bar(
    name="Regular Clients",
    x=client_summary["zone"],
    y=client_summary["total_clients"] - client_summary["star_clients"] - client_summary["new_clients"],
    marker_color="#95a5a6"
))

fig.add_trace(go.Bar(
    name="Star Clients",
    x=client_summary["zone"],
    y=client_summary["star_clients"],
    marker_color="#f1c40f"
))

fig.add_trace(go.Bar(
    name="New Clients",
    x=client_summary["zone"],
    y=client_summary["new_clients"],
    marker_color="#2ecc71"
))

fig.update_layout(
    title="Client Distribution by Zone and Type",
    barmode="stack",
    xaxis_title="Zone",
    yaxis_title="Number of Clients"
)
fig.show()

In [19]:
# Chart 6: Order Timeline (by issue date)
orders_by_date = all_orders_df.groupby(["issue_date", "status"]).size().reset_index(name="count")

fig = px.bar(
    orders_by_date,
    x="issue_date",
    y="count",
    color="status",
    title="Orders by Issue Date and Status",
    labels={"issue_date": "Issue Date", "count": "Number of Orders", "status": "Status"},
    color_discrete_map={"pending": "#e74c3c", "assigned": "#f1c40f", "delivered": "#2ecc71"}
)
fig.update_layout(xaxis_title="Issue Date", yaxis_title="Number of Orders")
fig.show()

## 8. Folium Map

Interactive map showing client distribution across Buenos Aires zones.

In [20]:
# Factory depot location (Quilmes)
DEPOT_LAT = -34.732296882352415
DEPOT_LON = -58.295903665460635

# Create base map centered on Buenos Aires
m = folium.Map(
    location=[DEPOT_LAT, DEPOT_LON],
    zoom_start=11,
    tiles="cartodbpositron"
)

# Add depot marker
folium.Marker(
    location=[DEPOT_LAT, DEPOT_LON],
    popup="<b>Eco-Bags Factory</b><br>Depot Location",
    tooltip="Factory Depot",
    icon=folium.Icon(color="black", icon="industry", prefix="fa")
).add_to(m)

# Add client markers
for _, client in all_clients_df.iterrows():
    zone_color = zone_color_map.get(client["zone_id"], "#808080")
    
    # Determine icon based on client type
    if client["is_star_client"]:
        icon_name = "star"
    elif client["is_new_client"]:
        icon_name = "plus"
    else:
        icon_name = "shopping-cart"
    
    popup_html = f"""
    <b>{client['business_name']}</b><br>
    Zone: {client['zone_id']}<br>
    Star Client: {'Yes' if client['is_star_client'] else 'No'}<br>
    New Client: {'Yes' if client['is_new_client'] else 'No'}
    """
    
    folium.CircleMarker(
        location=[client["latitude"], client["longitude"]],
        radius=8,
        popup=folium.Popup(popup_html, max_width=200),
        tooltip=client["business_name"],
        color=zone_color,
        fill=True,
        fill_color=zone_color,
        fill_opacity=0.7,
        weight=2
    ).add_to(m)

# Add legend
legend_html = """
<div style="position: fixed; bottom: 50px; left: 50px; z-index: 1000; 
            background-color: white; padding: 10px; border-radius: 5px;
            border: 2px solid grey; font-size: 12px;">
    <b>Zones</b><br>
    <i style="background: #FF6B6B; width: 12px; height: 12px; display: inline-block; border-radius: 50%;"></i> CABA<br>
    <i style="background: #4ECDC4; width: 12px; height: 12px; display: inline-block; border-radius: 50%;"></i> North Zone<br>
    <i style="background: #45B7D1; width: 12px; height: 12px; display: inline-block; border-radius: 50%;"></i> South Zone<br>
    <i style="background: #96CEB4; width: 12px; height: 12px; display: inline-block; border-radius: 50%;"></i> West Zone<br>
    <br><b>Depot</b><br>
    <i class="fa fa-industry" style="color: black;"></i> Factory
</div>
"""
m.get_root().html.add_child(folium.Element(legend_html))

# Display map
m

In [21]:
# Save map to output folder
map_output_path = project_root / "output" / "maps" / "client_distribution.html"
m.save(str(map_output_path))
print(f"Map saved to: {map_output_path}")

Map saved to: c:\Users\Santi\Desktop\CV\portafolio\Eco-Bags-Delivery-Optimizer\output\maps\client_distribution.html


## Summary

Phase 1 setup complete! The database now contains:

- **4 zones** (CABA, North, South, West)
- **79 localities** with coordinates across Buenos Aires
- **4 products** (3 main bag types + 1 special)
- **30 clients** distributed across zones
- **50 orders** with varied statuses and attributes

### Next Steps

1. **Phase 2**: Receipt extraction using LLM (OpenAI GPT-4o)
2. **Phase 3**: Priority scoring system implementation
3. **Phase 4**: Dispatch optimization with OR-Tools
4. **Phase 5**: Route optimization and visualization

In [22]:
# Final database statistics
print("=" * 50)
print("DATABASE SUMMARY")
print("=" * 50)

with db.get_session() as session:
    stats = {
        "Zones": session.query(ZoneModel).count(),
        "Localities": session.query(LocalityModel).count(),
        "Products": session.query(ProductModel).count(),
        "Clients": session.query(ClientModel).count(),
        "Orders": session.query(OrderModel).count(),
        "Order Items": session.query(OrderItemModel).count(),
        "Pending Orders": session.query(OrderModel).filter(OrderModel.status == "pending").count(),
        "Mandatory Orders": session.query(OrderModel).filter(OrderModel.is_mandatory == True).count(),
    }

for table, count in stats.items():
    print(f"{table}: {count}")

print("=" * 50)
print(f"Database location: {DB_PATH}")
print("Phase 1 complete!")

DATABASE SUMMARY
Zones: 4
Localities: 79
Products: 4
Clients: 30
Orders: 50
Order Items: 112
Pending Orders: 37
Mandatory Orders: 5
Database location: c:\Users\Santi\Desktop\CV\portafolio\Eco-Bags-Delivery-Optimizer\data\processed\delivery.db
Phase 1 complete!
